In [1]:
import pandas as pd
import numpy as np

import os
import sys

current = os.path.dirname(os.path.realpath("knn_peak_2"))
parent = os.path.dirname(current)

peaks = pd.read_csv(parent + "/player_data/ncaa_to_peak.csv")
pd.set_option("display.max_columns", 200)
peaks.head()

,prospect_age,ncaa_player_name,ncaa_team,ncaa_conf,ncaa_gp,ncaa_minutes_%,ncaa_ortg,ncaa_usg,ncaa_efg%,ncaa_ts%,ncaa_oreb%,ncaa_dreb%,ncaa_ast%,ncaa_to%,ncaa_ftm,ncaa_fta,ncaa_ft%,ncaa_2pm,ncaa_2pa,ncaa_2p%,ncaa_3pm,ncaa_3pa,ncaa_3p%,ncaa_blk%,ncaa_stl%,ncaa_ftr,ncaa_yr,ncaa_ht,ncaa_porpag,ncaa_adjoe,ncaa_foul_rate,ncaa_year,ncaa_recruit_rank,ncaa_ast_tov_ratio,ncaa_rim_makes,ncaa_rim_att,ncaa_mid_range_makes,ncaa_mid_range_att,ncaa_rim_fg%,ncaa_mid_fg%,ncaa_dunks_made,ncaa_dunks_att,ncaa_dunk_fg%,ncaa_draft_pick,ncaa_drtg,ncaa_adrtg,ncaa_dporpag,ncaa_stops,ncaa_bpm,ncaa_obpm,ncaa_dbpm,ncaa_gbpm,ncaa_min_per_game,ncaa_ogbpm,ncaa_dgbpm,ncaa_oreb,ncaa_dreb,ncaa_treb,ncaa_ast,ncaa_stl,ncaa_blk,ncaa_pts,ncaa_fga,ncaa_tov,ncaa_ppp,ncaa_rim_prop,ncaa_mid_prop,ncaa_3pt_prop,ncaa_ast_fg_ratio,GMM_Primary_Cluster,GMM_Secondary_Cluster,Full Role,HEIGHT,WEIGHT,STANDING REACH,WINGSPAN,STANDING VERTICAL,VERTICAL LEAP,LANE AGILITY,THREE QUARTER SPRINT,consensus_tier,peak_Player-additional,peak_Age,peak_G,peak_GS,peak_MP,peak_FG,peak_FGA,peak_FG%,peak_3P,peak_3PA,peak_3P%,peak_2P,peak_2PA,peak_2P%,peak_FT,peak_FTA,peak_FT%,peak_ORB,peak_DRB,peak_TRB,peak_AST,peak_STL,peak_BLK,peak_TOV,peak_PF,peak_PTS,peak_ORtg,peak_DRtg,peak_AST_2_TOV,peak_AST_2_FG,peak_PPP,peak_Season,peak_PER,peak_TS%,peak_3PAr,peak_FTr,peak_ORB%,peak_DRB%,peak_TRB%,peak_AST%,peak_STL%,peak_BLK%,peak_TOV%,peak_USG%,peak_OWS,peak_DWS,peak_WS,peak_WS/48,peak_OBPM,peak_DBPM,peak_BPM,peak_VORP,peak_Dist.,peak_2P_prop,peak_0-3_prop,peak_3-10_prop,peak_10-16_prop,peak_16-3P_prop,peak_3P_prop,peak_2P_fg%,peak_0-3_fg%,peak_3-10_fg%,peak_10-16_fg%,peak_16-3P_fg%,peak_3P_fg%,peak_2P_%astd,peak_3P_%astd,peak_dunk_%FGA,peak_dunk_#,peak_corner_%3PA,peak_corner_3P%,peak_heave_Att.,peak_heave_#
0,22,Richaun Holmes,Bowling Green,MAC,30,0.671,115.8,0.251,0.593,0.623,0.115,0.210,0.072,0.165,3.366667,4.733333,0.711,4.633333,7.900000,0.586,0.600000,1.433333,0.419,0.113,0.015,0.507,Sr,80,3.46191,122.240,3.4,2015,0.0,0.433325,3.566667,5.166667,1.066667,2.733333,0.6903,0.3902,1.500000,1.533333,0.9783,37.0,91.2274,91.5379,3.56104,192.802,7.98568,2.67221,5.31347,8.68190,28.7742,5.65496,3.02693,2.8710,5.1613,8.0323,0.8387,0.7097,2.6452,14.8065,9.333333,1.9355,1.108977,0.553571,0.292857,0.153571,0.089861,Skilled Forward,None,Skilled Forward,80.25,242.8,108.0,85.50,32.5,36.0,11.24,3.32,7.0,holmeri01,24.33,49.67,19.00,21.57,8.93,15.23,0.59,0.47,1.70,0.24,8.47,13.53,0.63,2.97,4.13,0.72,4.17,9.27,13.43,2.63,1.53,2.10,1.93,6.17,21.33,123.00,106.00,1.44,0.17,1.12,2017.33,18.57,0.63,0.11,0.27,9.50,21.00,15.27,8.57,1.53,3.73,10.30,16.93,2.40,1.47,3.83,0.17,0.17,0.87,1.03,0.80,6.77,0.89,0.52,0.22,0.10,0.05,0.11,0.63,0.76,0.44,0.47,0.31,0.16,0.77,0.67,0.26,78.00,0.10,0.16,0.67,0.00
1,23,Larry Nance Jr.,Wyoming,MWC,29,0.773,106.3,0.279,0.527,0.577,0.062,0.205,0.197,0.159,3.896552,4.965517,0.785,5.586207,10.551724,0.529,0.482759,1.413793,0.341,0.043,0.023,0.415,Sr,80,3.69410,117.739,2.6,2015,0.0,1.085692,3.206897,4.068966,2.379310,6.482759,0.7881,0.3670,1.586207,1.724138,0.9200,27.0,91.6832,91.8035,4.33077,203.468,5.38939,1.53945,3.84994,6.40162,34.9032,3.64650,2.75512,1.4194,5.8065,7.2258,2.4516,1.1935,1.1935,16.0645,11.965517,2.2581,0.979039,0.340058,0.541787,0.118156,0.204889,Skilled Forward,None,Skilled Forward,80.00,227.6,106.7,84.90,29.3,34.4,11.50,3.30,7.0,nancela02,27.00,66.00,19.33,23.19,7.17,12.67,0.57,0.50,1.53,0.28,6.70,11.10,0.60,2.57,3.73,0.69,4.60,9.73,14.33,4.30,2.60,1.23,1.97,5.27,17.40,126.33,108.67,2.21,0.35,1.07,2019.00,18.27,0.61,0.12,0.29,10.37,22.40,16.30,12.70,2.60,2.23,12.00,14.43,3.20,2.00,5.20,0.17,0.63,1.63,2.27,1.67,7.50,0.88,0.51,0.20,0.08,0.09,0.12,0.60,0.74,0.42,0.36,0.35,0.28,0.72,0.99,0.23,80.67,0.33,0.22,1.33,0.00
2,22,Jae Crowder,Marquette,BE,35,0.824,122.3,0.220,0.568,0.598,0.070,0.205,0.135,0.098,3.400000,4.628571,0.735,4.457143,7.400000,0.602,1.742857,5.057143,0.345,0.031,0.043,0.372,Sr,78,5.29599,132.030,3.2,2012,0.0,1.659136,3.657143,4.885714,0.800000,2.514286,0.7485,0.

After importing the data we need that contains the stats for player peaks, we need to decide which attributes we want to include as features in the PCA, which we will use for the KNN algorithm. 

Because we have a set of attributes we're predicting using our NCAA to Peak Random Forest, we must select from there, while also being mindful to only use what serves as useful. Because of this, we will select the following attributes.

In [2]:
knn_pca_attributes = ["prospect_age", "ncaa_usg", "ncaa_oreb%", "ncaa_dreb%", "ncaa_ast%", "ncaa_to%",
                      "ncaa_3p%", "ncaa_blk%", "ncaa_stl%", "ncaa_ast_tov_ratio", "ncaa_rim_fg%", "ncaa_mid_fg%",
                      "ncaa_dgbpm", "ncaa_ppp", "ncaa_rim_prop", "ncaa_mid_prop", "ncaa_3pt_prop", "STANDING REACH",
                      "VERTICAL LEAP", "LANE AGILITY", "consensus_tier"
                     ]

peaks_X = peaks.loc[:,knn_pca_attributes]

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
peaks_X = scaler.fit_transform(peaks_X)

peaks_pca = PCA()
peaks_pca_X = peaks_pca.fit_transform(peaks_X)
np.cumsum(peaks_pca.explained_variance_ratio_)

array([0.31202268, 0.42740338, 0.51335525, 0.5897413 , 0.66098649,
       0.71614749, 0.7662127 , 0.80679602, 0.84529214, 0.87290074,
       0.89691551, 0.91898974, 0.93760398, 0.95508571, 0.96834851,
       0.97796109, 0.98586662, 0.99232123, 0.9978003 , 0.99945995,
       1.        ])

We don't want to give components that have minimal difference between them (in variance explained) more value, so first we'll stop once the variance explained >70% (6).

In [21]:
peaks_pca = PCA(n_components = 6)
peaks_pca_X = pd.DataFrame(peaks_pca.fit_transform(peaks_X))

In [22]:
from sklearn.neighbors import NearestNeighbors
nba_neighbors = NearestNeighbors(n_neighbors=7)
nba_neighbors.fit(peaks_pca_X)

NearestNeighbors(n_neighbors=7)

In [23]:
import joblib
joblib.dump(scaler, "prospect_models/knn_models/knn_scaler.sav")
joblib.dump(peaks_pca, "prospect_models/knn_models/knn_pca.sav")
joblib.dump(nba_neighbors, "prospect_models/knn_models/knn_players.sav")

['prospect_models/knn_models/knn_players.sav']